# 布林带策略

In [11]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed

# 基础函数
import utilsJ

## 策略主体

### 买入信号
收盘价向上突破布林带上界或下界时买入。

### 买入仓位
逢信号就买入，每次买入1手。

### 卖出信号
收盘价向下突破布林带上界或下界时卖出

### 卖出仓位
清仓。

In [6]:
class Bollinger1(bt.Strategy):
    
    params = (
        ('printlog', False),
        ('bollinger_per', 20),
        ('bollinger_dev', 2),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Keep a reference to the "close" line, buyprice and sellprice in the data[0] dataseries
        self.dataclose = self.data.close
        self.buyprice = None
        self.sellprice = None
        
        # Add Bollinger Bands indicator
        self.lines.top = btind.BollingerBands(self.data,
                                              period=self.p.bollinger_per, 
                                              devfactor=self.p.bollinger_dev).top
        self.lines.bot = btind.BollingerBands(self.data,
                                              period=self.p.bollinger_per, 
                                              devfactor=self.p.bollinger_dev).bot

        # Create buy & sell signal
        self.buy_s = bt.Or(bt.And(self.dataclose(0) > self.lines.top(0), 
                                  self.dataclose(-1) < self.lines.top(-1)),
                           bt.And(self.dataclose(0) > self.lines.bot(0), 
                                  self.dataclose(-1) < self.lines.bot(-1)))
        
        self.sell_s = bt.Or(bt.And(self.dataclose(0) < self.lines.top(0), 
                                  self.dataclose(-1) > self.lines.top(-1)),
                           bt.And(self.dataclose(0) < self.lines.bot(0), 
                                  self.dataclose(-1) > self.lines.bot(-1)))


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.buyprice = order.executed.price

            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
        if self.order:
            return
        
        # Bollinger Buy Signal
        if self.buy_s[0]:
            self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % 
                     (self.dataclose[0], 100, self.getposition(self.data).size))
            self.order = self.buy(size = 100)

        # Bollinger Sell Singal
        elif self.sell_s[0] and self.getposition(self.data).size > 0:
            self.log('SELL CREATE (Close), Price: %.2f, Lots: %i' % 
                        (self.dataclose[0], self.getposition(self.data).size))
            self.order = self.close()

# 回测

In [7]:
s_date = datetime.datetime.now() - datetime.timedelta(days = 365)
e_date = datetime.datetime.now()
stock_code = '002057.SZ'
token = '74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6'


if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(Bollinger1, printlog = True) 

    # Create stock Data Feed
    if os.path.exists('.\\Data\\'+stock_code+'.csv'):
        df = pd.read_csv('.\\Data\\'+stock_code+'.csv', 
                    converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    else:
        df = utilsJ.stock_tushare(token, stock_code, s_date, e_date)

    data = btfeed.PandasData(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)
    
    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(10000)

    # Set leverage
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2021-11-29: BUY CREATE, Price: 9.33, Lots: 100, Current Position: 0
2021-11-30: BUY EXECUTED, Price: 9.33, Lot:100, Cash: 9066, Value: 9968
2021-11-30: SELL CREATE (Close), Price: 9.02, Lots: 100
2021-12-01: SELL EXECUTED, Price: 9.02, Lot:100, Cash: 9968, Value: 9968
2021-12-01: OPERATION PROFIT, GROSS -31.53, NET -31.53
2022-01-28: BUY CREATE, Price: 7.89, Lots: 100, Current Position: 0
2022-02-07: BUY EXECUTED, Price: 7.89, Lot:100, Cash: 9179, Value: 9980
2022-02-28: BUY CREATE, Price: 9.30, Lots: 100, Current Position: 100
2022-03-01: BUY EXECUTED, Price: 9.30, Lot:100, Cash: 8248, Value: 10085
2022-03-01: SELL CREATE (Close), Price: 9.19, Lots: 200
2022-03-02: SELL EXECUTED, Price: 9.19, Lot:200, Cash: 10085, Value: 10085
2022-03-02: OPERATION PROFIT, GROSS 117.28, NET 117.28
2022-03-16: BUY CREATE, Price: 7.89, Lots: 100, Current Position: 0
2022-03-17: BUY EXECUTED, Price: 7.89, Lot:100, Cash: 9296, Value: 10124
2022-04-25: SELL CREATE (Close)